In [1]:
# Preparação dos dados
import os

%matplotlib inline
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score

sns.set_theme()

In [2]:
## Importar dataset teste

data_path = '../data/' if os.path.exists(
    '../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

test_path = data_path + 'treated/prepared/numeric/no-dates/dataset-numeric-low-tensions-test-no-dates.csv'

test_df = pd.read_csv(test_path)

In [3]:
## Importar dataset treino
train_path = data_path + 'treated/prepared/numeric/no-dates/dataset-numeric-low-tensions-train-no-dates.csv'

train_df = pd.read_csv(train_path)

In [4]:
train_df.describe()

,energia_ativa_baixa_tensao_kwh,cpes_domestico_baixa_tensao,cpes_iluminacao_publica_baixa_tensao,cpes_nao_domestico_baixa_tensao,cpes_outros_baixa_tensao,cpes_mobilidade_eletrica_nao_baixa_tensao,cpes_mobilidade_eletrica_sim_baixa_tensao,densidade_populacional_pessoas_km2
count,4.641000e+03,4641.000000,4641.000000,4641.000000,4641.000000,4641.000000,4641.000000,4641.000000
mean,6.652473e+06,18795.518854,214.017453,4239.124111,129.467787,26681.424477,9.026287,314.542125
std,1.185535e+07,29390.997527,194.044633,6867.617704,206.654356,45554.097416,23.831036,856.169267
min,2.059940e+05,1042.000000,7.000000,174.000000,1.000000,1144.000000,0.000000,4.000000
25%,1.310791e+06,4581.000000,91.000000,1101.000000,21.000000,5940.000000,2.000000,25.000000
50%,2.685664e+06,8936.000000,141.000000,1989.000000,54.000000,11104.000000,2.000000,66.000000
75%,7.426747e+06,21405.000000,280.000000,4892.000000,157.000000,29350.000000,6.000000,177.000000
max,1.633819e+08,311824.000000,1184.000000,79956.000000,2186.000000,528158.000000,368.000000,7310.000000


In [5]:
test_df.describe()

,energia_ativa_baixa_tensao_kwh,cpes_domestico_baixa_tensao,cpes_iluminacao_publica_baixa_tensao,cpes_nao_domestico_baixa_tensao,cpes_outros_baixa_tensao,cpes_mobilidade_eletrica_nao_baixa_tensao,cpes_mobilidade_eletrica_sim_baixa_tensao,densidade_populacional_pessoas_km2
count,5.460000e+02,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000,546.000000
mean,5.381401e+06,18918.045788,215.278388,4274.082418,119.930403,33234.115385,14.593407,314.542125
std,1.106368e+07,29635.004603,195.630206,6925.569730,197.548003,50925.096260,35.708189,856.862054
min,1.000680e+05,1042.000000,7.000000,182.000000,1.000000,1892.000000,0.000000,4.000000
25%,8.825985e+05,4572.750000,91.000000,1112.000000,18.000000,8386.000000,2.000000,25.000000
50%,1.962897e+06,8928.500000,142.000000,2001.500000,48.000000,15531.000000,4.000000,66.000000
75%,5.025964e+06,21518.250000,281.000000,4927.250000,141.750000,39917.000000,10.000000,177.000000
max,1.641353e+08,312095.000000,1185.000000,79984.000000,2055.000000,530262.000000,388.000000,7310.000000


# Algoritmos de Aprendizagem supervisionada

## Dataset não normalizado

In [6]:
regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, solver='auto'),
    'Lasso': Lasso(alpha=1.0), 
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5),
    'k-NN': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100),
    'SVM': SVR(kernel='linear', max_iter=1000000000),
    'MLP': MLPRegressor(hidden_layer_sizes=(8,4), max_iter=200000)
}

In [7]:
cv_scores = pd.Series({
    name: np.mean(cross_val_score(regressor, train_df.drop(columns=['energia_ativa_baixa_tensao_kwh']), train_df['energia_ativa_baixa_tensao_kwh'], cv=5))
    for name, regressor in regressors.items()
})

cv_scores

/Users/caoma/Documents/Programming/GIT/Projeto-DECD/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.559e+15, tolerance: 5.907e+13
  model = cd_fast.enet_coordinate_descent(
/Users/caoma/Documents/Programming/GIT/Projeto-DECD/.venv/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.965e+15, tolerance: 5.907e+13
  model = cd_fast.enet_coordinate_descent(


Linear Regression    0.944744
Ridge                0.944744
Lasso                0.944744
ElasticNet           0.944751
k-NN                 0.929297
Decision Tree        0.845837
Random Forest        0.936781
SVM                  0.951433
MLP                  0.941973
dtype: float64

In [8]:
best = cv_scores.sort_values(ascending=False).index[0]
best_regressor = regressors[best]
best_regressor.fit(train_df.drop(columns=['energia_ativa_baixa_tensao_kwh']), train_df['energia_ativa_baixa_tensao_kwh'])

print(f'### {best} ###')

y_pred = best_regressor.predict(test_df.drop(columns=['energia_ativa_baixa_tensao_kwh']))

print('r2: {}'.format(r2_score(test_df['energia_ativa_baixa_tensao_kwh'], y_pred)))
print('mse: {}'.format(mean_squared_error(test_df['energia_ativa_baixa_tensao_kwh'], y_pred)))
print('mae: {}'.format(mean_absolute_error(test_df['energia_ativa_baixa_tensao_kwh'], y_pred)))

### SVM ###
r2: 0.6787181817616937
mse: 39254473085991.016
mae: 2798339.5838791123


### Ordenar os exemplos do conjunto de teste por ordem decrescente do erro da previsão do regressor e verificar se existe algum padrão relevante. 


In [9]:
df_error = test_df.copy()
df_error['error'] = np.abs(df_error['energia_ativa_baixa_tensao_kwh'] - y_pred)

df_error.sort_values('error', ascending=False).head(20)

,energia_ativa_baixa_tensao_kwh,cpes_domestico_baixa_tensao,cpes_iluminacao_publica_baixa_tensao,cpes_nao_domestico_baixa_tensao,cpes_outros_baixa_tensao,cpes_mobilidade_eletrica_nao_baixa_tensao,cpes_mobilidade_eletrica_sim_baixa_tensao,densidade_populacional_pessoas_km2,error
219,49701737,312095,1185,79939,2050,530262,388,5466,7.758240e+07
433,21242743,173756,938,40704,473,236910,104,1215,4.035582e+07
218,164135303,312074,1185,79984,2055,516364,382,5466,3.706299e+07
373,25851440,139500,707,34819,997,288380,92,5753,3.085388e+07
372,81748684,139499,707,34810,1000,286406,82,5753,2.511211e+07
507,20742447,138873,1181,22422,1005,245714,120,1821,2.432439e+07
31,11622919,96690,531,18709,572,171990,84,2544,2.198287e+07
223,11978360,89988,563,19734,526,168622,122,1215,2.137236e+07
99,13249506,86590,797,18398,937,144366,110,1071,2.012195e+07
47,8554055,81435,256,16812,334,122264,54,7310,1.950238e+07


## Dataset normalizado

In [10]:
## Importar dataset treino normalizado
trainn_path = data_path + 'treated/prepared/numeric/normalized/train/dataset-numeric-low-tensions-2223-no-dates-zscore.csv'

train_n_df = pd.read_csv(trainn_path)

## Importar dataset teste normalizado
testnn_path = data_path + 'treated/prepared/numeric/normalized/test/dataset-numeric-low-tensions-2024-no-dates-zscore.csv'

test_n_df = pd.read_csv(testnn_path)

In [11]:
normalized_regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0, solver='auto'),
    'Lasso': Lasso(alpha=1.0), 
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5),
    'k-NN': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100),
    'SVM': SVR(kernel='linear', max_iter=1000000000),
    'MLP': MLPRegressor(hidden_layer_sizes=(8,4), max_iter=200000)
}

In [12]:
cv_scores_n = pd.Series({
    name: np.mean(cross_val_score(regressor, train_n_df.drop(columns=['energia_ativa_baixa_tensao_kwh']), train_n_df['energia_ativa_baixa_tensao_kwh'], cv=5))
    for name, regressor in normalized_regressors.items()
})

cv_scores_n

Linear Regression    0.944744
k-NN                 0.907763
Decision Tree        0.911607
Random Forest        0.941423
SVM                  0.947686
MLP                  0.954123
dtype: float64

In [13]:
best = cv_scores_n.sort_values(ascending=False).index[0]
best_regressor_normalized = normalized_regressors[best]
best_regressor_normalized.fit(train_n_df.drop(columns=['energia_ativa_baixa_tensao_kwh']), train_n_df['energia_ativa_baixa_tensao_kwh'])

print(f'### {best} ###')

y_pred = best_regressor_normalized.predict(test_n_df.drop(columns=['energia_ativa_baixa_tensao_kwh']))

print('r2: {}'.format(r2_score(test_n_df['energia_ativa_baixa_tensao_kwh'], y_pred)))
print('mse: {}'.format(mean_squared_error(test_n_df['energia_ativa_baixa_tensao_kwh'], y_pred)))
print('mae: {}'.format(mean_absolute_error(test_n_df['energia_ativa_baixa_tensao_kwh'], y_pred)))

### MLP ###
r2: 0.711042891842121
mse: 0.28895710815787895
mae: 0.25459544545560076


### Ordenar os exemplos do conjunto de teste por ordem decrescente do erro da previsão do regressor e verificar se existe algum padrão relevante. 


In [14]:
df_error = test_n_df.copy()
df_error['error'] = np.abs(df_error['energia_ativa_baixa_tensao_kwh'] - y_pred)

df_error.sort_values('error', ascending=False).head(20)

,energia_ativa_baixa_tensao_kwh,cpes_domestico_baixa_tensao,cpes_iluminacao_publica_baixa_tensao,cpes_nao_domestico_baixa_tensao,cpes_outros_baixa_tensao,cpes_mobilidade_eletrica_nao_baixa_tensao,cpes_mobilidade_eletrica_sim_baixa_tensao,densidade_populacional_pessoas_km2,error
219,4.009605,9.901999,4.961457,10.935462,9.779089,9.768929,10.466759,6.017517,6.470255
218,14.362264,9.901290,4.961457,10.941965,9.804422,9.495768,10.298576,6.017517,3.940453
433,1.434955,5.229624,3.697713,5.265029,1.788899,4.003186,2.506108,1.051842,2.955818
507,1.389694,4.051457,4.940992,2.622825,4.484385,4.176226,2.954595,1.759722,2.579557
373,1.851898,4.072634,2.515831,4.414500,4.443851,5.014813,2.169743,6.352767,2.532791
372,6.908851,4.072600,2.515831,4.413199,4.459051,4.976015,1.889438,6.352767,2.531754
99,0.711817,2.285608,2.976304,2.041257,4.139849,2.184263,2.674291,0.883633,1.768501
31,0.564662,2.626734,1.615349,2.086204,2.290502,2.727204,1.945499,2.604273,1.705804
120,4.347910,2.986571,2.080939,2.214254,2.118234,3.170614,5.309154,2.197767,1.662984
223,0.596818,2.400375,1.779073,2.234342,2.057434,2.661007,3.010656,1.051842,1.662974
